# Test on mountain car

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import gym

In [2]:
from vae import VAE, create_decoder, create_encoder
from transition import TransitionModel
from agent import DAIFAgent

In [3]:
enc = create_encoder(2, 2)
dec = create_decoder(2, 2)
tran = TransitionModel(2, 1)

# create the prior distribution
# prior_observations = tf.convert_to_tensor(np.array([0.5, 100]).astype("float"))
# prior = tfp.distributions.MultivariateNormalDiag(loc=prior_observations, scale_diag=np.ones_like())

daifa = DAIFAgent("test", enc, dec, tran, 10, 100, 100)

Metal device set to: Apple M1 Pro


2022-07-07 06:11:59.968729: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-07 06:11:59.968933: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Test training the models

In [4]:
from util import random_observation_sequence, transform_observations

In [5]:
env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

o, a, r = random_observation_sequence(env, 10000)
o_scaled = transform_observations(o)
o_scaled

TypeError: transform_observations() missing 1 required positional argument: 'noise_stddev'

In [6]:
x_train = np.concatenate([o_scaled[0:len(o_scaled)-1], a], axis=1)
y_train_mean = o_scaled[1:]
y_train_stddev = np.ones_like(y_train_mean)

x_train

NameError: name 'o_scaled' is not defined

In [7]:
tran.compile(optimizer=tf.keras.optimizers.Adam())

tran.fit(x_train, (y_train_mean, y_train_stddev), batch_size=10, epochs=30)

NameError: name 'x_train' is not defined

## Test the VAE model

In [8]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
tran = TransitionModel(2, 1)

# create the prior distribution
# prior_observations = tf.convert_to_tensor(np.array([0.5, 100]).astype("float"))
# prior = tfp.distributions.MultivariateNormalDiag(loc=prior_observations, scale_diag=np.ones_like())

daifa = DAIFAgent("test", enc, dec, tran, 10, 100)

In [9]:
daifa.model_vae.fit(y_train_mean, batch_size=10, epochs=10)

NameError: name 'y_train_mean' is not defined

In [10]:
batch = 100
dim = 40
a = np.random.random((batch, dim))
a = tf.convert_to_tensor(a)

b = np.ones((batch, dim))
b = tf.convert_to_tensor(b)
rl = keras.losses.binary_crossentropy(a, b)


kl_loss = -0.5 * (1 + b - tf.square(a) - tf.exp(b))
kl_loss = tf.reduce_sum(kl_loss, axis=1)

print(rl, kl_loss)

tf.Tensor(
[8.45834744 6.97253126 7.45176529 7.39323376 8.84239961 8.0194043
 7.24296769 8.56139991 7.11050548 7.02522314 9.1205567  7.06234085
 6.78191898 6.34084347 8.00939453 8.51033093 6.73068413 7.05090782
 6.85602492 8.15828836 8.2529087  7.95415945 7.98247929 7.27710926
 8.29394296 7.91625148 7.585719   8.64476301 8.10830271 7.24324656
 7.8024236  8.40839054 8.35463476 7.7523882  7.12641877 7.60879971
 7.08634624 8.54674745 7.75737687 7.1623217  7.63549979 8.82123039
 8.20043625 8.09783095 7.15048313 8.22300395 7.51502318 7.15922819
 7.52419056 7.8112862  6.99882264 7.7441865  8.1488129  7.80719543
 7.42767147 6.61341895 7.40759462 7.5097976  7.33212411 7.80905072
 8.78588039 5.81200295 7.81632576 7.04220643 8.59628931 8.60405681
 7.52813975 6.74359781 7.2027611  7.3612393  7.97566669 7.73609674
 7.88444022 7.97568048 7.1631057  6.66873631 7.8421183  8.40946401
 8.5077804  8.36701256 7.35211195 7.58369218 7.59417093 7.72655829
 7.66202095 7.46983171 7.92347258 6.31128362 8.04326

## Action Selection

Test the FEEF stuff

In [11]:
mean_best_policies = tf.zeros(3)
std_best_policies = tf.ones(3)

policy_distr = tfp.distributions.MultivariateNormalDiag(loc=mean_best_policies, scale_diag=std_best_policies)

ps = tf.reduce_sum(policy_distr.sample(10), axis=1)

print(ps)

neg_ps = -1*ps

print(neg_ps)

result = tf.math.top_k(neg_ps, k=3, sorted=False)

print(result)

#
# result = tf.math.top_k(neg_FEEF_sum, self.n_policy_candidates, sorted=False)
# min_FEEF_indices = result.indices
#
# mean_best_policies = policies[:, min_FEEF_indices].mean(1)
# std_best_policies = policies[:, min_FEEF_indices].std(1)

tf.Tensor(
[-0.96444476  0.9348421   0.32520223  0.30133337 -0.5702816   0.85975087
 -1.0687883  -2.9884071  -1.444941   -1.6965554 ], shape=(10,), dtype=float32)
tf.Tensor(
[ 0.96444476 -0.9348421  -0.32520223 -0.30133337  0.5702816  -0.85975087
  1.0687883   2.9884071   1.444941    1.6965554 ], shape=(10,), dtype=float32)
TopKV2(values=<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2.9884071, 1.6965554, 1.444941 ], dtype=float32)>, indices=<tf.Tensor: shape=(3,), dtype=int32, numpy=array([7, 9, 8], dtype=int32)>)


In [12]:
tf.zeros((10,10))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [14]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
tran = TransitionModel(2, 1)

# create the prior distribution
# prior_observations = tf.convert_to_tensor(np.array([0.5, 100]).astype("float"))
# prior = tfp.distributions.MultivariateNormalDiag(loc=prior_observations, scale_diag=np.ones_like())

daifa = DAIFAgent(None, enc, dec, tran)

env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

o, a, r = random_observation_sequence(env, 10000)

o_scaled = transform_observations(o, [0.1, 0.05])
o_scaled

# Perception
daifa.model_vae.fit(o_scaled, batch_size=200, epochs=30)
z_mean, z_stddev, z = daifa.model_vae.encoder(o)

recon = daifa.model_vae.decoder(z)

# transition
z_mean, z_stddev, z = daifa.model_vae.encoder(o)

# input
z_train = np.concatenate([z_mean[0:len(z_mean) - 1], a], axis=1)

# output
z_plus_one_train_mean = o[1:]
z_plus_one_train_stddev = z_stddev[1:]
daifa.tran.fit(z_train, (z_plus_one_train_mean, z_plus_one_train_stddev), batch_size=100, epochs=50)

Epoch 1/30


2022-07-07 06:12:57.292089: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-07 06:12:57.423384: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 7ms/step - loss: 0.7602 - reconstruction_loss: 0.6785 - kl_loss: 0.0758
Epoch 2/30
5/5 [==============================] - 0s 7ms/step - loss: 0.7293 - reconstruction_loss: 0.6756 - kl_loss: 0.0495
Epoch 3/30
5/5 [==============================] - 0s 7ms/step - loss: 0.7056 - reconstruction_loss: 0.6724 - kl_loss: 0.0298
Epoch 4/30
5/5 [==============================] - 0s 7ms/step - loss: 0.6889 - reconstruction_loss: 0.6703 - kl_loss: 0.0162
Epoch 5/30
5/5 [==============================] - 0s 7ms/step - loss: 0.6775 - reconstruction_loss: 0.6684 - kl_loss: 0.0076
Epoch 6/30
5/5 [==============================] - 0s 8ms/step - loss: 0.6700 - reconstruction_loss: 0.6664 - kl_loss: 0.0029
Epoch 7/30
5/5 [==============================] - 0s 7ms/step - loss: 0.6668 - reconstruction_loss: 0.6654 - kl_loss: 7.7720e-04
Epoch 8/30
5/5 [==============================] - 0s 7ms/step - loss: 0.6653 - reconstruction_loss: 0.6648 - kl_loss: 1.3243e-04
Epo

2022-07-07 06:12:58.872920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 7ms/step - kl_loss: 0.4317
Epoch 3/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.3275
Epoch 4/50
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.2600
Epoch 5/50
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.2147
Epoch 6/50
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.1803
Epoch 7/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.1556
Epoch 8/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.1354
Epoch 9/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.1188
Epoch 10/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.1046
Epoch 11/50
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0919
Epoch 12/50
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0811
Epoch 13/50
10/10 [==============================] - 0s 7ms/step - kl_loss: 0.0714
Epoch 14/50
10/10 [====

In [16]:
daifa.cem_policy_optimisation(o_scaled[0])
# policy = list(daifa.select_action(z_plus_one_train_mean[0]))
# policy

(<tf.Tensor: shape=(15,), dtype=float32, numpy=
 array([-0.20070818,  0.7253131 , -0.8150349 , -0.4245601 , -0.8902022 ,
        -0.6787055 , -0.73471665, -0.11469162, -0.7634379 , -0.5958372 ,
         0.73066735, -0.09735839, -0.8139771 , -0.22439788, -0.39049834],
       dtype=float32)>,
 <tf.Tensor: shape=(15,), dtype=float32, numpy=
 array([0.1196427 , 0.5445143 , 0.0685593 , 0.30862826, 0.15756068,
        0.31297576, 0.11279172, 0.2131243 , 0.09952834, 0.14200002,
        0.51246357, 0.18465346, 0.09358477, 0.21129449, 0.26233423],
       dtype=float32)>)

In [297]:
env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

observation, info = env.reset()
print(observation, info)

policy = daifa.select_action(z_plus_one_train_mean[0]).reshape(10, 1)
policy

# execute the policy
for action in policy:

    observation, reward, done, info = env.step(action)

    print(observation, reward, done, info)

    if done:
        observation, info = env.reset(return_info=True)

env.close()

-0.4614573 0.0
[49499.965 49499.996 49500.04  49500.047 49500.023 49500.02  49500.008
 49499.953 49500.004 49499.984 49499.92  49500.062 49499.992 49500.008
 49499.926 49499.965 49500.02  49499.984 49500.008 49500.043 49500.043
 49500.04  49499.96  49499.94  49500.008 49499.918 49499.99  49500.055
 49500.03  49499.926 49499.953 49499.96  49499.992 49499.957 49500.016
 49500.01  49499.984 49499.98  49500.027 49499.992 49500.016 49500.055
 49500.082 49499.98  49500.043 49500.047 49499.97  49499.99  49499.953
 49500.03  49499.953 49499.953 49499.91  49499.977 49499.977 49500.047
 49499.996 49500.06  49500.043 49499.938 49499.984 49500.016 49499.984
 49499.984 49499.96  49499.957 49499.98  49500.074 49500.008 49499.996
 49499.973 49500.062 49499.938 49499.953 49500.016 49499.984 49499.992
 49500.06  49499.883 49500.004 49500.023 49500.008 49500.05  49499.93
 49499.95  49499.945 49499.957 49499.914 49499.938 49499.98  49499.99
 49499.945 49500.035 49500.05  49499.992 49499.906 49499.934 494

In [287]:
env.action_space.sample()

array([0.36845294], dtype=float32)

In [28]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
tran = TransitionModel(2, 1)

# create the prior distribution
# prior_observations = tf.convert_to_tensor(np.array([0.5, 100]).astype("float"))
# prior = tfp.distributions.MultivariateNormalDiag(loc=prior_observations, scale_diag=np.ones_like())

daifa = DAIFAgent("test", enc, dec, tran, 10, 100)

In [29]:
env = gym.make('MountainCarContinuous-v0')
env.action_space.seed(42)

o, a, r = random_observation_sequence(env, 10000)
o_noisy = o + np.random.normal(loc=0.0, scale=[0.05, 0.005], size=o.shape)

o_scaled = transform_observations(o_noisy)
o_scaled

array([[0.47845995, 0.49991999],
       [0.51759177, 0.49046143],
       [0.50813442, 0.51365131],
       ...,
       [0.57395726, 0.50586725],
       [0.57064717, 0.46195592],
       [0.5754071 , 0.47639147]])

In [30]:
# Perception
daifa.model_vae.fit(o_scaled, batch_size=200, epochs=20)

Epoch 1/20
5/5 [==============================] - 0s 7ms/step - loss: 0.8552 - reconstruction_loss: 0.7133 - kl_loss: 0.1348
Epoch 2/20
5/5 [==============================] - 0s 8ms/step - loss: 0.8158 - reconstruction_loss: 0.7098 - kl_loss: 0.1001
Epoch 3/20
1/5 [=====>........................] - ETA: 0s - loss: 0.7876 - reconstruction_loss: 0.7061 - kl_loss: 0.0815

2022-07-06 05:54:16.754819: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 8ms/step - loss: 0.7811 - reconstruction_loss: 0.7061 - kl_loss: 0.0707
Epoch 4/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7549 - reconstruction_loss: 0.7050 - kl_loss: 0.0470
Epoch 5/20
5/5 [==============================] - 0s 8ms/step - loss: 0.7359 - reconstruction_loss: 0.7034 - kl_loss: 0.0293
Epoch 6/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7205 - reconstruction_loss: 0.7013 - kl_loss: 0.0170
Epoch 7/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7114 - reconstruction_loss: 0.7003 - kl_loss: 0.0090
Epoch 8/20
5/5 [==============================] - 0s 7ms/step - loss: 0.7035 - reconstruction_loss: 0.6986 - kl_loss: 0.0042
Epoch 9/20
5/5 [==============================] - 0s 7ms/step - loss: 0.6999 - reconstruction_loss: 0.6978 - kl_loss: 0.0017
Epoch 10/20
5/5 [==============================] - 0s 8ms/step - loss: 0.6978 - reconstruction_loss: 0.6968 - kl_loss: 6.0820e-04
Epoch 

In [31]:
z_mean, z_stddev, z = daifa.model_vae.encoder(o)

recon = daifa.model_vae.decoder(z)
print(recon)
print(o_scaled)

tf.Tensor(
[[0.5257379  0.5014833 ]
 [0.5014981  0.5088885 ]
 [0.5229919  0.49525228]
 ...
 [0.4788991  0.4971832 ]
 [0.5250801  0.5057    ]
 [0.49217275 0.4854029 ]], shape=(1000, 2), dtype=float32)
[[0.47845995 0.49991999]
 [0.51759177 0.49046143]
 [0.50813442 0.51365131]
 ...
 [0.57395726 0.50586725]
 [0.57064717 0.46195592]
 [0.5754071  0.47639147]]


In [32]:
# transition
z_mean, z_stddev, z = daifa.model_vae.encoder(o)

# input
z_train = np.concatenate([z_mean[0:len(z_mean) - 1], a], axis=1)

# output
z_plus_one_train_mean = o[1:]
z_plus_one_train_stddev = z_stddev[1:]

In [33]:
daifa.tran.fit(z_train, (z_plus_one_train_mean, z_plus_one_train_stddev), batch_size=100, epochs=20)

Epoch 1/20
10/10 [==============================] - 0s 9ms/step - kl_loss: 0.3604
Epoch 2/20
 1/10 [==>...........................] - ETA: 0s - kl_loss: 0.3227

2022-07-06 05:54:22.898256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 7ms/step - kl_loss: 0.2692
Epoch 3/20
10/10 [==============================] - 0s 5ms/step - kl_loss: 0.2056
Epoch 4/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.1597
Epoch 5/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.1272
Epoch 6/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.1043
Epoch 7/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0873
Epoch 8/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0746
Epoch 9/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0644
Epoch 10/20
10/10 [==============================] - 0s 5ms/step - kl_loss: 0.0559
Epoch 11/20
10/10 [==============================] - 0s 6ms/step - kl_loss: 0.0486
Epoch 12/20
10/10 [==============================] - 0s 5ms/step - kl_loss: 0.0421
Epoch 13/20
10/10 [==============================] - 0s 5ms/step - kl_loss: 0.0366
Epoch 14/20
10/10 [====

In [34]:
# test action selection
daifa.tran(z_train)

[<tf.Tensor: shape=(999, 2), dtype=float32, numpy=
 array([[-0.5141744 , -0.00226609],
        [-0.51317906, -0.00211778],
        [-0.50906724, -0.0015824 ],
        ...,
        [-0.5140544 ,  0.00733696],
        [-0.51738137,  0.00694774],
        [-0.5164505 ,  0.00712375]], dtype=float32)>,
 <tf.Tensor: shape=(999, 2), dtype=float32, numpy=
 array([[0.861043  , 1.0008498 ],
        [0.8625715 , 1.0007256 ],
        [0.8687212 , 1.0001794 ],
        ...,
        [0.8321117 , 1.0099009 ],
        [0.82790154, 1.0103225 ],
        [0.8303419 , 1.0101273 ]], dtype=float32)>]

In [35]:
z_plus_one_train_mean

array([[-5.63812613e-01, -7.72724088e-05],
       [-5.63636005e-01,  1.76615955e-04],
       [-5.61897516e-01,  1.73848611e-03],
       ...,
       [-3.47172916e-01, -2.90909829e-03],
       [-3.52811098e-01, -5.63819287e-03],
       [-3.60669792e-01, -7.85868429e-03]])

In [37]:
z_plus_one_train_stddev

<tf.Tensor: shape=(999, 2), dtype=float32, numpy=
array([[0.86313003, 1.0035411 ],
       [0.86315316, 1.0034735 ],
       [0.8634986 , 1.0030464 ],
       ...,
       [0.93430805, 1.0015261 ],
       [0.9326206 , 1.0020291 ],
       [0.9301086 , 1.0024189 ]], dtype=float32)>

In [152]:
a = np.array([1,2])

np.stack([a]*10).shape

(10, 2)

In [89]:
a = np.zeros((100, 1, 3))
b = np.ones((100, 1, 3))

obs = np.ones((100, 1, 2))

p = np.random.normal(loc=0, scale=1, size=(100, 10))

new_obs = np.concatenate([obs, (p[:, 0]).reshape(100,1,1)], axis=2)

new_obs.shape

join = np.hstack([a, b, b, b, b, new_obs])
print(join.shape)
# np.concatenate([join, (p[:, 0]).reshape(100,1,1)], axis=2).shape

(100, 6, 3)


In [43]:
z_plus_one_train_mean[0].reshape(1,2)

array([[-5.63812613e-01, -7.72724088e-05]])

### Run on the env

In [11]:
env = gym.make('MountainCar-v0')
env.action_space.seed(42)

observation, info = env.reset()
print(observation, info)

for _ in range(1000):

    policy = daifa.select_action(observation)

    # execute the policy
    for _ in range(10):

    observation, reward, done, info = env.step()

    print(observation, reward, done, info)

    if done:
        observation, info = env.reset(return_info=True)

env.close()

IndentationError: expected an indented block (2468360782.py, line 14)